# Fully Connected Neural Network (FCNN) Architectures

## In this Notebook is the framework for four different FCNN models. Here they can be editted and changed for use in main program.

In [ ]:
# Import Required Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Define input_dim, the number of possible genes

# Define the FCNN Class Called TLS_Identifier_CNN
# This First FCNN is Simple, 4 total layers, 2 hidden layers, with ReLU, Drop out and Sigmoid Binary Classification.
# Number of Hidden Layers:
#     Too few layers: The model may be unable to capture complex patterns.
#     Too many layers: The model may overfit or become computationally expensive.

# Test combinations of layers and neurons. For example:

# Layers: [1, 2, 3]
# Neurons: [64, 128, 256]
# Example combinations:

# 1 layer with 64 neurons.
# 2 layers with 128 → 64 neurons.
# 3 layers with 256 → 128 → 64 neurons.
class Simple_FCNN(nn.Module):
    def __init__(self, input_size):
      super(Simple_FCNN, self).__init__()

      # Define Input Layer which will pass information to hidden layers. Going from n-number of Genes to 128 neurons
      self.fc1 = nn.Linear(input_dim, 128)  # Fully connected layer
      self.relu = nn.ReLU()                 # ReLU (Rectified Linear Unit) activation for non-linearity

      # Hidden Layer 1 to Hidden Layer 2 (64 neurons)
      # Creating drop out layer to hopefully avoid overfitting by randomly dropping neurons throughout training
      self.fc2 = nn.Linear(128, 64)        # Fully connected layer
      self.dropout = nn.Dropout(0.3)       # Dropout for regularization

      # Hidden Layer 2 to Output Layer (1 neuron)
      # Applying signmoid layer to output for final binary classification, TLS(1), or NO_TLS(0)
      self.fc3 = nn.Linear(64, 1)           # Fully connected layer (1 neuron for binary classification)
      self.sigmoid = nn.Sigmoid()           # Sigmoid activation for binary classification

    def forward(self, x):
      # Define the forward pass
      x = self.relu(self.fc1(x))            # Apply ReLU after first layer
      x = self.dropout(x)                   # Apply Dropout
      x = self.relu(self.fc2(x))            # Apply ReLU after second layer
      x = self.dropout(x)                   # Apply Dropout again
      x = self.sigmoid(self.fc3(x))         # Apply Sigmoid for final probability output
      return x

# Creating another FCNN called Deep_FCNN which has 4 layers and increased amounts of neurons
# This model will be more computationally expensive
class Deep_FCNN(nn.Module):
    def __init__(self, input_dim):
      super(Deep_FCNN, self).__init__()
      self.fc1 = torch.nn.Linear(input_dim, 256)  # Increase neurons
      self.fc2 = torch.nn.Linear(256, 128)
      self.fc3 = torch.nn.Linear(128, 64)
      self.fc4 = torch.nn.Linear(64, 1)
      self.relu = torch.nn.ReLU()
      self.sigmoid = torch.nn.Sigmoid()
      self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, x):
      x = self.relu(self.fc1(x))
      x = self.dropout(x)
      x = self.relu(self.fc2(x))
      x = self.dropout(x)
      x = self.relu(self.fc3(x))
      x = self.dropout(x)
      x = self.sigmoid(self.fc4(x))
      return x


# Creating another FCNN called BatchNorm_FCNN which is similar to deep FCNN, but adds a batch normalization layer after each hidden layer
# Normalizes activations during training, which can accelerate convergence
# Reduces internal covariate shift, improving generalization
class BatchNorm_FCNN(torch.nn.Module):
    def __init__(self, input_dim):
      super(BatchNorm_FCNN, self).__init__()
      self.fc1 = torch.nn.Linear(input_dim, 256)
      self.bn1 = torch.nn.BatchNorm1d(256)  # Batch normalization, after head hidden layer
      self.fc2 = torch.nn.Linear(256, 128)
      self.bn2 = torch.nn.BatchNorm1d(128)
      self.fc3 = torch.nn.Linear(128, 64)
      self.bn3 = torch.nn.BatchNorm1d(64)
      self.fc4 = torch.nn.Linear(64, 1)
      self.relu = torch.nn.ReLU()
      self.sigmoid = torch.nn.Sigmoid()
      self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, x):
      x = self.relu(self.bn1(self.fc1(x)))  # Batch normalization after fc1, and again after each layer
      x = self.dropout(x)
      x = self.relu(self.bn2(self.fc2(x)))
      x = self.dropout(x)
      x = self.relu(self.bn3(self.fc3(x)))
      x = self.dropout(x)
      x = self.sigmoid(self.fc4(x))
      return x

# Creating another FCNN called Residual_FCNN
# Adding residual connections to the model, allowing the network to learn idenity mappig
# This may help mitigate vanishing gradient in the deeper network
# Allowing the model to retain learned patterns from earlier layers
class Residual_FCNN(torch.nn.Module):
    def __init__(self, input_dim):
      super(Residual_FCNN, self).__init__()
      self.fc1 = torch.nn.Linear(input_dim, 256)
      self.fc2 = torch.nn.Linear(256, 128)
      self.fc3 = torch.nn.Linear(128, 64)
      self.fc4 = torch.nn.Linear(64, 1)
      self.relu = torch.nn.ReLU()
      self.sigmoid = torch.nn.Sigmoid()
      self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, x):
      # Applying a linear transformation to the input to match the dimensions of fc2 output
      # This creates a new "identity" that has the correct dimensions
      identity = self.fc1(x)
      identity = self.relu(identity) # Applying activation to match post-activation output
      identity = self.dropout(identity) # Applying regularization, and matching fc2 input dimension
      identity = self.fc2(identity)
      # Performing the rest of the forward pass as before
      x = self.relu(self.fc1(x))
      x = self.dropout(x)
      # Adding the modified "identity" here which now has the correct dimensions
      x = self.relu(self.fc2(x) + identity)  # Residual connection
      x = self.dropout(x)
      x = self.relu(self.fc3(x))
      x = self.dropout(x)
      x = self.sigmoid(self.fc4(x))
      return x